Loading the gRASPA library
==========================

In [1]:
import gRASPA as g

# Running gRASPA, the original taste with no modifications
==========================
You can set the cycles here, it will overwrite the values read from `simulation.input` file

In [2]:
W = g.Initialize()
W.NumberOfInitializationCycles = 100
W.NumberOfProductionCycles = 0
g.RUN(W)
g.finalize(W)

/home/z/anaconda3/bin/python3.12------------------GENERAL SIMULATION SETUP-------------
Finished Checking Number of Components, There are 1 framework, 1 Adsorbates, 2 total Components
DONE Reading Model Info from simulation.input file
Running Cycles in the Normal Way
-------------------------------------------------------
device_random[0] = 2.30000 4.50000 6.70000
====== Preparing Simulation box 0 ======
Parsing [1] Component
-------------- READING AdsorbateComponent 0 (CO2) --------------
ACCUMULATED Probabilities:
Translation Probability:      0.25000
Rotation Probability:         0.50000
Special Rotation Probability: 0.50000
Widom Probability:            0.50000
Reinsertion Probability:      0.75000
Identity Swap Probability:    0.75000
CBCF Swap Probability:        0.75000
Swap Probability:             1.00000
Volume Probability:           1.00000
Gibbs Swap Probability:       1.00000
Gibbs Volume Probability:     1.00000
Sum of Probabilities:         1.00000
-------------- END OF 

================FUGACITY COEFFICIENT CALCULATION================
Checking: Current Fugacity Coeff for 1 component: 1.00000
Every Adsorbate Component has fugacity coefficient assigned, skip EOS calculation!
----------------- MEMORY ALLOCAION STATUS -----------------
System allocate_sizes are: 2304, 2000
Component allocate_sizes are: 2304, 2000
Allocated Blocksum size: 3601, vdw_real size: 3601, fourier_size: 0
------------------------------------------------------------
------------------- SIMULATION BOX PARAMETERS -----------------
Pressure:        0.00060
Box Volume:      42656.20098
Box Beta:        0.00404
Box Temperature: 298.00000
---------------------------------------------------------------
======================== CALCULATING INITIAL STAGE ENERGY ========================
****** Calculating VDW + Real Energy (CPU) ******
Host-Host   VDW: 0.00000; Real: 0.00000
Host-Guest  VDW: 0.00000; Real: 0.00000
Guest-Guest VDW: 0.00000; Real: 0.00000
********** PRINTING COMPONENT ENERGIES*

# Running gRASPA, with your modifications
====================
## To add your modifications, the user needs to modify the gRASPA processes
  * To change what happens during an MC move, one needs to decompose `g.RUN()` function and add their modifications there.
  * Users are given control of these intermediate variables, such as acceptance ratio, selected component/molecule, which move to perform

In [3]:
# run the single-box MC moves through python #
box_index = 0 # run simulation box 0 #

In [4]:
# Initialize MC move, this step will initialize a stage of the simulation (Initialization/Equilibration/Production phases)
# For example, it will initialize (clean up) the vectors for storing averages if the stage is production phase
g.InitializeMC(W, box_index)

==  RUNNING PRODUCTION PHASE   ==
CBMC Uses 10 trial positions and 10 trial orientations


In [5]:
# Run a move, starting by randomly selecting a component in the box and a molecule in that component
g.Select_Box_Component_Molecule(W, box_index)

In [6]:
# Select a move, here we tell the code to run a single-particle insertion move (no CBMC)
MoveType = g.MoveTypes.SINGLE_INSERTION
W.SystemComponents[box_index].MCMoveVariables.MoveType = int(MoveType) # write it back to the variables so that later processes will know

In [7]:
W.SystemComponents[box_index].MCMoveVariables.MoveType

2

## Prepare the MC move
* Then to run the new move, first we do the preparation of the mc move
* This includes: 
  * book-keeping the amount of MC move performed
  * initialize trial positions for the insertion move, etc.
* :memo: We can see from the random number offset on the GPU that the preparation of the insertion move costs 3 x double3 of random numbers
  * This corresponds to the random positions in x, y, and z directions

In [8]:
# Prepare the MC move, single particle insertion
print(f"Initial Random offset: {W.Random.offset}\n")
g.SingleBody_Prepare(W, systemId = box_index)
print(f"After Random offset: {W.Random.offset}\n")

Initial Random offset: 28202

After Random offset: 28205



## Calculation part of the MC move
This part includes calculating the vdw+real part of the trial energy, ewald summation, tail correction, and acceptance ratio (un-biased)
a `MoveEnergy` is generated, and you can check the deltaE of this move by `DeltaE.print()` and check the singleparticle move Pacc

In [9]:
# Move Calculation #
DeltaE = g.SingleBody_Calculation(W, systemId = box_index)
DeltaE.print()

HHVDW: 0.00000, HHReal: 0.00000, HGVDW: 6448.46706, HGReal: 500.24272, GGVDW: -344.32224, GGReal: 98.20524, HHEwaldE: 0.00000,
 HGEwaldE: -365.77690,
 GGEwaldE: 17.75031, TailE: 0.00000, DNN_E: 0.00000
Stored HGVDW: 0.00000, Stored HGReal: 0.00000, Stored HGEwaldE: 0.00000


In [10]:
W.SystemComponents[box_index].MCMoveVariables.Overlap
W.SystemComponents[box_index].MCMoveVariables.Pacc

3.922521744437901e-14

# Move Acceptance #
* Dictates whether a move is accepted by Metropolis scheme
* You can then check acceptance by looking at the temporary variables

In [11]:
# Move Acceptance #
comp = W.SystemComponents[box_index].MCMoveVariables.component
print(f"number of molecules: {W.SystemComponents[box_index].NumberOfMolecules[comp]}")
g.SingleBody_Acceptance(W, box_index, DeltaE)
print(f"Accept?: {W.SystemComponents[box_index].MCMoveVariables.Accept}\n")
print(f"number of molecules: {W.SystemComponents[box_index].NumberOfMolecules[comp]}")

number of molecules: 18
Accept?: False

number of molecules: 18


## That move was rejected and number of molecules stays the same 
* We can see Pacc is small
* DeltaE has a really high HGVDW value, indicating unfavorable host-guest interactions
## What if we force the move to be accepted?
* We can modify this by changing the Pacc to something big

In [12]:
# The move is rejected, what if we can change that manually? #
W.SystemComponents[box_index].MCMoveVariables.Pacc = 0.95
g.SingleBody_Acceptance(W, box_index, DeltaE)
print(f"Accept?: {W.SystemComponents[box_index].MCMoveVariables.Accept}\n")
print(f"number of molecules: {W.SystemComponents[box_index].NumberOfMolecules[comp]}")

Accept?: True

number of molecules: 19


## Now the move is forced to be accepted!